# 1.0. Initialize

In [1]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
from nibabel.viewers import OrthoSlicer3D
from nibabel import nifti1
import nibabel as nib
from matplotlib import pylab as plt
import pandas as pd

from ml_logic.data import load_nii_from_gcp

In [2]:
from google.cloud import storage
import os


client = storage.Client()

#get the Bucket Name from .env
bucket_name = os.getenv('BUCKET_NAME')
#set bucket
bucket = client.get_bucket(bucket_name)
#give the blob_name that you want to access
blob_name='BraTS20_Training_005_seg.nii' 
#get the blob(file)
blob = bucket.blob(blob_name)
# make a lockal cach folder
directory = "cache" 
parent_dir = "/home/yaoyx001/code/" #need to change the yaoyx001 to your username
# Path
path = os.path.join(parent_dir, directory)
  
# Create the directory
if not os.path.exists(path):
    os.makedirs(path)
    print("Directory '% s' created" % directory)

cache_file = os.path.join(path, blob_name)
#save the file in cache_folder

if not os.path.isfile(cache_file):
    blob.download_to_filename(cache_file)



img = load_nii_from_gcp('BraTS20_Training_006_seg.nii',"/home/yaoyx001/code/cache")

# 1.1 Select Data

# 1.2. Model pipeline construction

In [4]:
df = pd.read_csv('raw_data/name_mapping_2020.csv')

df = df.drop(columns=['BraTS_2017_subject_ID', 'BraTS_2018_subject_ID',
                        'BraTS_2019_subject_ID', 'TCGA_TCIA_subject_ID'])
df['Grade'].value_counts()

HGG    293
LGG     76
Name: Grade, dtype: int64

In [5]:
df['t1_nii']= df['BraTS_2020_subject_ID'].apply(lambda x: np.array(load_nii_from_gcp(x+'_t1.nii',path).dataobj))

In [6]:
df_test = df.iloc[5,:].copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369 entries, 0 to 368
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Grade                  369 non-null    object
 1   BraTS_2020_subject_ID  369 non-null    object
 2   t1_nii                 369 non-null    object
dtypes: object(3)
memory usage: 8.8+ KB


In [10]:
df['t1_nii'].shape

(369,)

In [7]:
df.iloc[0, 2].valu

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [11]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"
    # The path to your file to upload
    # source_file_name = "local/path/to/file"
    # The ID of your GCS object
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    # Optional: set a generation-match precondition to avoid potential race conditions
    # and data corruptions. The request to upload is aborted if the object's
    # generation number does not match your precondition. For a destination
    # object that does not yet exist, set the if_generation_match precondition to 0.
    # If the destination object already exists in your bucket, set instead a
    # generation-match precondition using its generation number.
    generation_match_precondition = 0

    blob.upload_from_filename(source_file_name, if_generation_match=generation_match_precondition)

    print(
        f"File {source_file_name} uploaded to {destination_blob_name}."
    )

In [12]:

df.to_pickle("raw_data/Grade_ID_t1_nii.pkl")  


In [13]:
upload_blob(bucket_name='row_data_bucket', source_file_name="raw_data/Grade_ID_t1_nii.pkl",destination_blob_name="Grade_ID_t1_nii.pkl")



File raw_data/Grade_ID_t1_nii.pkl uploaded to Grade_ID_t1_nii.pkl.
